<a href="https://colab.research.google.com/drive/1m4txj2euGGplKLIpnA0IzGsD9FJ9uSCF?usp=sharing"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table Of Contents

- <a href='#0'>0. Install Turicreate </a>  

- <a href='#1'>1. Load the dataset</a>

- <a href='#2'>2. Building the Recommendation Engine</a>

  - <a href='#21'>2.1. Recommendations based on Scaled/Normalized PurchaseCount</a>

    - <a href='#211'>2.1.1. Sample Recommendations</a>
    - <a href='#212'> 2.1.2. Model Evaluation</a>

  - <a href='#22'>2.2. Recommendations based on Non-Scaled PurchaseCount</a>

    - <a href='#221'>2.2.1. Sample Recommendations</a>
    - <a href='#222'>2.2.2. Model Evaluation</a>


- <a href='#3'>3. Conclusion</a>

---

# <a id='0'> 0. Install Turicreate </a>

In [ ]:

!pip install turicreate

     |████████████████████████████████| 92.0MB 55kB/s 
     |████████████████████████████████| 327kB 50.0MB/s 
     |████████████████████████████████| 3.4MB 46.3MB/s 
     |████████████████████████████████| 86.4MB 61kB/s 
     |████████████████████████████████| 450kB 48.4MB/s 
     |████████████████████████████████| 3.8MB 46.2MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
  Created wheel for resampy: filename=resampy-0.2.1-cp36-none-any.whl size=320848 sha256=d98a21d4062198ee886688b53ca304d3d30665edcd1a443abefd2910d2915247
  Stored in directory: /root/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=56ba3e50be9ce92408ae0af2fe3e85d30f1125a002c85cde7f799791d8308148
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built resampy gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, 

# <a id='1'> 1. Load the Dataset </a>

In [ ]:
import numpy as np
import pandas as pd
import turicreate as tc

In [ ]:
df = pd.read_csv("RecommenderEngine.csv", )

In [ ]:
df.head()

,CustomerKey,EnglishProductSubcategoryName,FirstName,LastName,MiddleName,Number of Records,ProductSubcategoryAlternateKey
0,21768,Road Bikes,Cole,Watson,A,1,2
1,28389,Mountain Bikes,Rachael,Martinez,M,1,1
2,25863,Mountain Bikes,Sydney,Wright,S,1,1
3,14501,Road Bikes,Ruben,Prasad,NaN,1,2
4,11003,Mountain Bikes,Christy,Zhu,NaN,1,1


In [ ]:
df_custName = df['FirstName'] +' '+ df['MiddleName'] +' '+ df['LastName']

---
# <a id='2'> 2. Building the Recommendation Engine </a>

In [ ]:
# Creating the dataset

df2 = pd.DataFrame(data = df[['CustomerKey', 'ProductSubcategoryAlternateKey', 'EnglishProductSubcategoryName']])
df2['CustomerName'] = df_custName.copy()
df2.head()

,CustomerKey,ProductSubcategoryAlternateKey,EnglishProductSubcategoryName,CustomerName
0,21768,2,Road Bikes,Cole A Watson
1,28389,1,Mountain Bikes,Rachael M Martinez
2,25863,1,Mountain Bikes,Sydney S Wright
3,14501,2,Road Bikes,NaN
4,11003,1,Mountain Bikes,NaN


In [ ]:
df2.to_csv('CustomerProduct_Dataset.csv', index=False)

In [ ]:
df = pd.read_csv("CustomerProduct_Dataset.csv", )
df.head()

,CustomerKey,ProductSubcategoryAlternateKey,EnglishProductSubcategoryName,CustomerName
0,21768,2,Road Bikes,Cole A Watson
1,28389,1,Mountain Bikes,Rachael M Martinez
2,25863,1,Mountain Bikes,Sydney S Wright
3,14501,2,Road Bikes,NaN
4,11003,1,Mountain Bikes,NaN


In [ ]:
df.ProductSubcategoryAlternateKey.value_counts()

37    17332
2      8068
28     7981
31     6440
1      4970
21     3332
19     2190
3      2167
30     2121
20     1430
22     1019
29      908
32      733
23      568
25      562
26      328
27      249
Name: ProductSubcategoryAlternateKey, dtype: int64

In [ ]:
subCatCount = df.ProductSubcategoryAlternateKey.value_counts()
# subCat = subCatCount.index
# PurchaseCount = subCatCount.values

In [ ]:
df_subCat = pd.DataFrame({'ProductSubcategoryAlternateKey':subCatCount.index, 'PurchaseCount':subCatCount.values,})

In [ ]:
df_final = df.drop_duplicates(keep='first', inplace=False)
df_final.shape

(44734, 4)

In [ ]:
df_final.head()

,CustomerKey,ProductSubcategoryAlternateKey,EnglishProductSubcategoryName,CustomerName
0,21768,2,Road Bikes,Cole A Watson
1,28389,1,Mountain Bikes,Rachael M Martinez
2,25863,1,Mountain Bikes,Sydney S Wright
3,14501,2,Road Bikes,NaN
4,11003,1,Mountain Bikes,NaN


In [ ]:
df_final_merge = df_final.merge(df_subCat, on='ProductSubcategoryAlternateKey')
df_final_merge.head()

,CustomerKey,ProductSubcategoryAlternateKey,EnglishProductSubcategoryName,CustomerName,PurchaseCount
0,21768,2,Road Bikes,Cole A Watson,8068
1,14501,2,Road Bikes,NaN,8068
2,27645,2,Road Bikes,Colin G Anand,8068
3,16624,2,Road Bikes,Albert R Alvarez,8068
4,27621,2,Road Bikes,NaN,8068


In [ ]:
df_custKeyName = pd.DataFrame(data = df_final_merge[['CustomerKey', 'CustomerName']])
df_custKeyName.drop_duplicates(keep='first', inplace=True)

In [ ]:
subCatNames = df_final_merge[['ProductSubcategoryAlternateKey', 'EnglishProductSubcategoryName']]
df_subCatNames = subCatNames.drop_duplicates(keep='first', inplace=False)
df_subCatNames

,ProductSubcategoryAlternateKey,EnglishProductSubcategoryName
0,2,Road Bikes
6397,1,Mountain Bikes
10486,28,Bottles and Cages
15034,3,Touring Bikes
17177,31,Helmets
23137,37,Tires and Tubes
31627,27,Bike Stands
31870,21,Jerseys
35062,20,Gloves
36438,30,Fenders


In [ ]:
df_subCatNames.shape

(17, 2)

In [ ]:
df_final_merge.to_csv('ProductCountDataset.csv', index=False)
actions = tc.SFrame.read_csv('ProductCountDataset.csv')

Finished parsing file /content/ProductCountDataset.csv

Parsing completed. Parsed 100 lines in 0.120041 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,str,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /content/ProductCountDataset.csv

Parsing completed. Parsed 44734 lines in 0.068077 secs.

### <a id='21'> 2.1. Recommendations based on Scaled/Normalized PurchaseCount </a>

In [ ]:
# Standardizing ProductCount

from sklearn.preprocessing import StandardScaler

df_final_merge_scaled = df_final_merge.copy()

scalar = StandardScaler(with_mean=False)
scalar.fit(df_final_merge_scaled.drop(['CustomerKey','ProductSubcategoryAlternateKey',
                                'EnglishProductSubcategoryName', 'CustomerName'],axis=1))
scaled = scalar.transform(df_final_merge_scaled.drop(['CustomerKey','ProductSubcategoryAlternateKey',
                                                              'EnglishProductSubcategoryName', 'CustomerName'],
                                                             axis=1))

df_final_merge_scaled['PurchaseCount'] = scaled
df_final_merge_scaled.head(10)

,CustomerKey,ProductSubcategoryAlternateKey,EnglishProductSubcategoryName,CustomerName,PurchaseCount
0,21768,2,Road Bikes,Cole A Watson,1.474914
1,14501,2,Road Bikes,NaN,1.474914
2,27645,2,Road Bikes,Colin G Anand,1.474914
3,16624,2,Road Bikes,Albert R Alvarez,1.474914
4,27621,2,Road Bikes,NaN,1.474914
5,27616,2,Road Bikes,Emma W Brown,1.474914
6,20042,2,Road Bikes,Brad D Deng,1.474914
7,16351,2,Road Bikes,Martha C Xu,1.474914
8,16517,2,Road Bikes,Katrina M Raji,1.474914
9,27606,2,Road Bikes,Courtney A Edwards,1.474914


In [ ]:
df_final_merge_scaled.to_csv('ScaledProductCountDataset.csv', index=False)
ScaledActions = tc.SFrame.read_csv('ScaledProductCountDataset.csv')

Finished parsing file /content/ScaledProductCountDataset.csv

Parsing completed. Parsed 100 lines in 0.111688 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,str,str,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /content/ScaledProductCountDataset.csv

Parsing completed. Parsed 44734 lines in 0.071855 secs.

In [ ]:
ScaledTraining_data, ScaledValidation_data = tc.recommender.util.random_split_by_user(ScaledActions,'CustomerKey',
                                                                          'ProductSubcategoryAlternateKey')
ScaledModel = tc.recommender.create(ScaledTraining_data, 'CustomerKey', 'ProductSubcategoryAlternateKey',
                              target= 'PurchaseCount')

Preparing data set.

Data has 44275 observations with 18429 users and 17 items.

Data prepared in: 0.111588s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Convergence on objective within bounds.

Computing final objective value and training RMSE.

Final objective value: 3.10972e-05

Final training RMSE: 0.00536669

In [ ]:
r = ScaledModel.recommend(users=[28203], k=5)

In [ ]:
r

CustomerKey,ProductSubcategoryAlternateKey ...,score,rank
28203,37,1.4264815588625668,1
28203,2,1.1484586418750284,2
28203,26,1.0683089521857498,3
28203,1,1.0596652113976477,4
28203,32,1.0359873612555264,5


In [ ]:
def recommendProducts(s):
  name = df_custKeyName.loc[df_custKeyName['CustomerKey'] == s, 'CustomerName'].iloc[0]
  print(s,' Customer Name: ', name, ', Recommendations:\n')
  results = ScaledModel.recommend(users=[s], k=7)
  for i in results['ProductSubcategoryAlternateKey']:
    print(df_subCatNames.loc[df_subCatNames.ProductSubcategoryAlternateKey == i,
                             'EnglishProductSubcategoryName'].values[0])

#### <a id='211'> 2.1.1. Sample Recommendations </a>

In [ ]:
recommendProducts(11449) # recommendProducts(CustomerID)

11449  Customer Name:  Alvin E Hu , Recommendations:

Road Bikes
Bike Racks
Helmets
Cleaners
Hydration Packs
Bike Stands
Bottles and Cages


In [ ]:
recommendProducts(28204)

28204  Customer Name:  Peter K Xu , Recommendations:

Tires and Tubes
Road Bikes
Mountain Bikes
Cleaners
Bike Racks
Bottles and Cages
Hydration Packs


---
#### <a id='212'> 2.1.2. Model Evaluation </a>

In [ ]:
ScaledEval = ScaledModel.evaluate(ScaledValidation_data)
ScaledEval


Precision and recall summary statistics by cutoff
+--------+---------------------+---------------------+
| cutoff |    mean_precision   |     mean_recall     |
+--------+---------------------+---------------------+
|   1    |  0.2698412698412699 | 0.23434744268077604 |
|   2    | 0.22486772486772486 | 0.39616402116402116 |
|   3    | 0.17283950617283947 |  0.4497354497354497 |
|   4    | 0.15145502645502654 |  0.5207231040564374 |
|   5    |  0.1375661375661375 |  0.5899470899470898 |
|   6    |  0.1252204585537919 |  0.6360229276895945 |
|   7    | 0.11640211640211642 |  0.6924603174603174 |
|   8    | 0.10681216931216932 |  0.7224426807760143 |
|   9    | 0.10052910052910057 |   0.76521164021164  |
|   10   | 0.09503233392122282 |  0.8004850088183421 |
+--------+---------------------+---------------------+
[10 rows x 3 columns]


Overall RMSE: 0.15629240602687172

Per User RMSE (best)
+-------------+------------------------+-------+
| CustomerKey |          rmse          | count |
+

{'precision_recall_by_user': Columns:
 	CustomerKey	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 6804
 
 Data:
 +-------------+--------+-----------+--------+-------+
 | CustomerKey | cutoff | precision | recall | count |
 +-------------+--------+-----------+--------+-------+
 |    27671    |   1    |    0.0    |  0.0   |   1   |
 |    27671    |   2    |    0.0    |  0.0   |   1   |
 |    27671    |   3    |    0.0    |  0.0   |   1   |
 |    27671    |   4    |    0.0    |  0.0   |   1   |
 |    27671    |   5    |    0.0    |  0.0   |   1   |
 |    27671    |   6    |    0.0    |  0.0   |   1   |
 |    27671    |   7    |    0.0    |  0.0   |   1   |
 |    27671    |   8    |    0.0    |  0.0   |   1   |
 |    27671    |   9    |    0.0    |  0.0   |   1   |
 |    27671    |   10   |    0.0    |  0.0   |   1   |
 +-------------+--------+-----------+--------+-------+
 [6804 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_ro

---
### <a id='22'> 2.2. Recommendations based on Non-Scaled PurchaseCount </a>

In [ ]:
training_data, validation_data = tc.recommender.util.random_split_by_user(actions,'CustomerKey',
                                                                          'ProductSubcategoryAlternateKey')

model = tc.recommender.create(training_data, 'CustomerKey', 'ProductSubcategoryAlternateKey',
                              target= 'PurchaseCount')

Preparing data set.

Data has 44275 observations with 18429 users and 17 items.

Data prepared in: 0.119584s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 108673

Final training RMSE: 319.12

In [ ]:
def recommendProducts(s):
  name = df_custKeyName.loc[df_custKeyName['CustomerKey'] == s, 'CustomerName'].iloc[0]
  print(s,' Customer Name: ', name, ', Recommendations:\n')
  results = model.recommend(users=[s], k=7)
  for i in results['ProductSubcategoryAlternateKey']:
    print(df_subCatNames.loc[df_subCatNames.ProductSubcategoryAlternateKey == i,
                             'EnglishProductSubcategoryName'].values[0])

##### <a id='221'> 2.2.1. Sample Recommendations </a>

In [ ]:
recommendProducts(11449) # recommendProducts(CustomerID)

11449  Customer Name:  Alvin E Hu , Recommendations:

Jerseys
Hydration Packs
Helmets
Fenders
Bike Stands
Bottles and Cages
Bike Racks


In [ ]:
recommendProducts(28203)

28203  Customer Name:  Victor I Gomez , Recommendations:

Road Bikes
Socks
Gloves
Bottles and Cages
Jerseys
Mountain Bikes
Caps


---
#### <a id='222'> 2.2.2. Model Evaluation </a>

In [ ]:
eval = model.evaluate(validation_data)
eval


Precision and recall summary statistics by cutoff
+--------+---------------------+---------------------+
| cutoff |    mean_precision   |     mean_recall     |
+--------+---------------------+---------------------+
|   1    | 0.17724867724867724 | 0.15057319223985902 |
|   2    | 0.16931216931216936 | 0.28990299823633164 |
|   3    | 0.13315696649029984 |  0.3353174603174603 |
|   4    | 0.11706349206349206 | 0.38977072310405636 |
|   5    |  0.1068783068783069 | 0.45458553791887113 |
|   6    | 0.09788359788359788 |  0.5008818342151675 |
|   7    | 0.09372637944066514 |  0.5553350970017638 |
|   8    | 0.09027777777777776 |  0.6086860670194004 |
|   9    | 0.08818342151675487 |  0.667768959435626  |
|   10   | 0.08736037624926513 |  0.7277336860670196 |
+--------+---------------------+---------------------+
[10 rows x 3 columns]


Overall RMSE: 502.3140425260383

Per User RMSE (best)
+-------------+--------------------+-------+
| CustomerKey |        rmse        | count |
+----------

{'precision_recall_by_user': Columns:
 	CustomerKey	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 6804
 
 Data:
 +-------------+--------+---------------------+--------+-------+
 | CustomerKey | cutoff |      precision      | recall | count |
 +-------------+--------+---------------------+--------+-------+
 |    27671    |   1    |         0.0         |  0.0   |   1   |
 |    27671    |   2    |         0.0         |  0.0   |   1   |
 |    27671    |   3    |         0.0         |  0.0   |   1   |
 |    27671    |   4    |         0.25        |  1.0   |   1   |
 |    27671    |   5    |         0.2         |  1.0   |   1   |
 |    27671    |   6    | 0.16666666666666666 |  1.0   |   1   |
 |    27671    |   7    | 0.14285714285714285 |  1.0   |   1   |
 |    27671    |   8    |        0.125        |  1.0   |   1   |
 |    27671    |   9    |  0.1111111111111111 |  1.0   |   1   |
 |    27671    |   10   |         0.1         |  1.0   |   1   |
 +-------------+--

---
# <a id='3'> 3. Conclusion </a>

- Model built on Scaled/Normalized PurchaseCount has RMSE: 0.158

- Model built on Non-Scaled PurchaseCount has overall RMSE: 488.01 

- Since the model built on Scaled/Normalized PurchaseCount has very good RMSE value, we choose it for our Recommendation Engine